In [ ]:
import pandas as pd
import math
import operator
import warnings
warnings.filterwarnings('ignore')
from sklearn.externals import joblib

In [ ]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
size_mapping = {"pv":0,"buy":1,'fav':0,'cart':0}
data["action"] = data["action"].map(size_mapping)
user_click=data.groupby('userid').itemid.apply(list).to_dict()

In [ ]:
#可以把这个函数去掉，用1代替
def base_contribute_score():
    '''
    item cf bse sim contribution score
    '''
    return 1

In [ ]:
co_appear = {}#电影的共现矩阵
item_user_click_time = {}  # 记录这个item被多少用户点击过，用于分母
for user, itemlist in user_click.items():
    for index_i in range(0, len(itemlist)):
        #记录list中的元素
        itemid_i = itemlist[index_i]
        #将itemid作为key，0为value.字典长度为该用户评分过的电影数量
        item_user_click_time.setdefault(itemid_i, 0)  
        #将该用户评分过的Item_id对应的value加1，表示movie被点击的次数
        item_user_click_time[itemid_i] += 1 
        for index_j in range(index_i + 1, len(itemlist)):
            itemid_j = itemlist[index_j]
            co_appear.setdefault(itemid_i, {})
             #字典   { 当前itemid :  {  ( itemid , 相似度  ),(),()....} }
            co_appear[itemid_i].setdefault(itemid_j, 0)
            co_appear[itemid_i][itemid_j] += base_contribute_score()

            co_appear.setdefault(itemid_j, {})
            co_appear[itemid_j].setdefault(itemid_i, 0)
            #理解不了这可以看《推荐系统实战》P54上面那个图
            co_appear[itemid_j][itemid_i] += base_contribute_score()

item_sim_score = {}
#排序后的 i2i
item_sim_score_sorted = {}
for itemid_i, relate_item in co_appear.items():
    for itemid_j, co_time in relate_item.items():
        sim_score = co_time / (
                    math.sqrt(item_user_click_time[itemid_i]) * math.sqrt(item_user_click_time[itemid_j]))
        item_sim_score.setdefault(itemid_i, {})
        item_sim_score[itemid_i].setdefault(itemid_j, 0)
        item_sim_score[itemid_i][itemid_j] = sim_score
#排序
for itemid in item_sim_score:
    item_sim_score_sorted[itemid] = sorted(item_sim_score[itemid].items(), key= \
        operator.itemgetter(1), reverse=True)

In [ ]:
#返回推荐列表
#data为召回的数据
def predict_list(data):
    x = data[['userid','itemid','categoryid']]
    #以概率形式输出预测值
    y_hat = LR.predict_proba(x)[:, 1]
    #将每个概率值与itemid一一对应
    result=pd.DataFrame({'ItemID':x.itemid,'Action':y_hat})
    temp=result.sort_values("Action",ascending=False)
    #删除重复出现的itemid
    res = temp.drop_duplicates("ItemID")
    #取前十个item推荐给用户
    pre_list = res.iloc[:10,0].tolist()
    return pre_list

In [ ]:
#召回50个 item
list_recall_50 = []
count = 0
LR=joblib.load('LR.model')
#创建空的DataFrame
recall_dataframe = pd.DataFrame(columns=['userid','itemid','categoryid','action','timestamp'])
#遍历test_set
for index,row in test.iterrows():
    #记录当前行的UserID
    item_id = row["ItemID"]
    #遍历 i2i表
    for key,value in item_2_item.items():
        #在i2i中找出与当前行test数据相同的ItemID
        if item_id == key:
            #取出相似度前50的ItemID
            for j in range(50):
                list_recall_50.append(value[j][0])
             #召回test_set中与当前item相似的所有数据 
            for j in list_recall_50:
                temp_df = data.loc[ (data["itemid"]==j)]
                #召回的数据
                recall_dataframe = recall_dataframe.append(temp_df,ignore_index=True)
#             print("recall_data.shape  = ",recall_dataframe.shape)
            #生成推荐列表    
            pred_list = predict_list(recall_dataframe)
            print("predict_list  = ",pred_list)
            #测试效果，如果test的当前 itemid 在推荐的10 item 中，就积一分
            if item_id in pred_list:
                count +=1
print("count  = ",count)#输出累计得分